In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'/Users/Farook/breast-cancer-'

In [4]:
!pip install dagshub

  Preparing metadata (setup.py) ... done
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.10.2-py3-none-any.whl.metadata (1.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 3.0 MB/s eta 0:00:0000:0100:01
Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
Using cached s3transfer-0.10.2-py3-none-any.whl (82 kB)
  Created wheel for fusepy: filename=fusepy-3.0.1-py3-none-any.whl size=10486 sha256=c7d1a75a5556b389f3fe907011f68d861f148afd44cca60331bb10265c4234a2
  Stored in directory: /Users/Farook/Library/Caches/pip/wheels/96/e4/a1/c2ff90a4d140b39eb2af34232c5fce1b59641e9da39d13ce75
Successfully built fusepy
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.5.0
    Uninstalling tenacity-8.5.0:
      Successfully uninstalled tenacity-8.5.0
  Attempting uninstall: rich
    Found existing installation: rich 13.7.1
    Uninstalling rich-13.7.1:
      Successfully uninstalled rich-13.7.1
  Attempting uninstall:

In [14]:
import dagshub
dagshub.init(repo_owner='furiousbird8', repo_name='breast-cancer-', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Initialized MLflow to track repo "furiousbird8/breast-cancer-"

Repository furiousbird8/breast-cancer- initialized!

2024/09/03 16:22:00 INFO mlflow.tracking._tracking_service.client: 🏃 View run secretive-grub-819 at: https://dagshub.com/furiousbird8/breast-cancer-.mlflow/#/experiments/0/runs/f6974474a35b4f679266308d71cb185b.
2024/09/03 16:22:00 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/furiousbird8/breast-cancer-.mlflow/#/experiments/0.


In [22]:
import tensorflow as tf
model= tf.keras.models.load_model("artifacts/training/model.h5")

In [29]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int


In [30]:
from breast.constants import *
from breast.utils.common import read_yaml, create_directories, save_json

In [31]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
        path_of_model="artifacts/training/model.h5",
        training_data="artifacts/data_ingestion/Data",
        mlflow_uri="https://dagshub.com/furiousbird8/breast-cancer-.mlflow",
        all_params=self.params,
        params_image_size=self.params.IMAGE_SIZE,
        params_batch_size=self.params.BATCH_SIZE
        )
        
        return eval_config

In [32]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [33]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [34]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 298 images belonging to 3 classes.
13/13 ━━━━━━━━━━━━━━━━━━━━ 122s 9s/step - accuracy: 0.9004 - loss: 0.3674


2024/09/03 17:22:49 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/09/03 17:23:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/09/03 17:23:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.
2024/09/03 17:23:34 INFO mlflow.tracking._tracking_service.client: 🏃 View run abundant-roo-900 at: https://dagshub.com/furiousbird8/breast-cancer-.mlflow/#/experiments/0/runs/b85e71f49b054a56b935b6cc9b1cf34d.
2024/09/03 17:23:34 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/furiousbird8/breast-cancer-.mlflow/#/experiments/0.
